# TaskB-2&3

In [1]:
# creator fkon0001@student.monash.edu
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
'''
Initial client
Use the fit5148_db Database
@climateCollection -> input the collection form taskA, the mongodb format climatePart-1.csv
@fireCollection -> input the collection form taskA, the mongodb format firePart-1.csv
@MergeCollection -> new database based on the document model I have designed in Task B1.
'''
client = MongoClient()
db = client.fit5148_db
climateCollection = db.climate1
fireCollection = db.fire1
MergeCollection = db.MergeCollection

In [3]:
'''
This fuction use for merging fireCollection and climateCollection
@collection -> is a list use for storing the merge-result
'''
def mergeClimateFire():
    cursorFire = fireCollection.find({})
    cursorClimate = climateCollection.find({})
    collection = []
    
    # put all document of climate collection into the collection list
    for eachClimate in cursorClimate:
        collection.append(eachClimate)
    
    # merge fire data at here
    for eachRecord in collection:
        # initial a fire field for each document
        eachRecord["Fire"] = []
        # for each climate document find whether exit fire document with same "Date" value
        for eachFire in fireCollection.find({"Date" : eachRecord["Date"]}):
            # delete the field "Date", because in climate document already has one, we do not need same field here
            del eachFire["Date"]
            # for good understanding change the "_id" of fire document to "_FireId"
            eachFire.update(_FireId = eachFire.pop("_id"))
            # add a new field "SurfaceTemperature" which has two field, "Celcius" & "kelvin".
            # these two field use for represent "Surface Temperature (Celcius)" and "Surface Temperature (kelvin)"
            eachFire.update(SurfaceTemperature = {"Celcius":eachFire['Surface Temperature (Celcius)'],'kelvin':eachFire['Surface Temperature (kelvin)']})
            # delete two field, because we already represent in "SurfaceTemperature"
            eachFire.pop('Surface Temperature (Celcius)')
            eachFire.pop('Surface Temperature (kelvin)')
            # add Fire document into climate data
            eachRecord["Fire"].append(eachFire)

    return collection


In [ ]:
# get collection of merging fireCollection and climateCollection
collection = mergeClimateFire()

In [ ]:
# check collection 
pprint(collection)

In [6]:
# insert the collection into new database based on the document model I have designed in Task B1
def insertCollection(collection, target):
    for eachRecord in collection:
        target.insert_one(eachRecord)
    return target
# for initial collection, make it convient to test
def deleteCollection(target):
    target.drop()

In [7]:
# delete first
deleteCollection(MergeCollection)

In [8]:
# insert here
insertCollection(collection, MergeCollection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'fit5148_db'), 'MergeCollection')

In [ ]:
def checkResult(collection):
    cursor = collection.find({})
    for eachDoc in cursor:
        pprint(eachDoc)

checkResult(MergeCollection)

# taskB-4
### For this task I choose TaskA-2

In [52]:
import multiprocessing as mp
from multiprocessing import Pool
# this fuction use to show the result directly, use for check whether the result is right    
def showResultDirectly(collection, date):   
    cursor = collection.find({"Date": date})
    for i in cursor:
        pprint(i)

# Look the result 
showResultDirectly(MergeCollection, "2017-12-15")

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Fire': [{'Confidence': 92,
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.627,
           'Longitude': 149.33,
           'Power': 23.1,
           'SurfaceTemperature': {'Celcius': 42, 'kelvin': 315.9},
           '_FireId': ObjectId('5af3cc31b14a6b797be553cb')},
          {'Confidence': 78,
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.658,
           'Longitude': 149.339,
           'Power': 17.3,
           'SurfaceTemperature': {'Celcius': 36, 'kelvin': 309.8},
           '_FireId': ObjectId('5af3cc31b14a6b797be553cc')},
          {'Confidence': 51,
           'Datetime': '2017-12-15T13:17:17',
           'Latitude': -37.623,
           'Longitude': 149.323,
           'Power': 17.8,
           'SurfaceTemperature': {'Celcius': 38, 'kelvin': 311.9},
           '_FireId': ObjectId('5af3cc31b14a6b797be553cd')},
          {'Confidence': 65,
           'Datetime': '2017-12-15

In [54]:
'''
Assume that we have 3 processors, as a result we can divde the data to 3 parts by using two time nodes. 
For divde equally, the time node shuold be the date of 10 and 20. If date less than 10 put it to first processor, else 
if it greater than 10 and less 20, put it to second processor, else put it to the last one.
'''
def whichProcessor(date):

    if int(date[-2:]) <= 10:
        return 0
    else:
        if int(date[-2:]) <= 20:
            return 1
        else:
            return 2
        
'''
This is the algorithm how we partioning data, for each record in the collection, we take the date value and compare
with the time node. This function will back a value to piont the exactly which processor
'''

def partionByTime(collection):
    result = []
    
    for i in range(3):
        result.append([])
        
    cursor = collection.find({})
    for eachRecord in cursor:
        result[whichProcessor(eachRecord["Date"])].append(eachRecord)
    return result


'''
We alread get a partioned data. When user input a date value, firstly we use "whichProcessor" to determine
which worker may has this value, then searh one by one in that worker. Everytime we find the value we put it to
result list.
'''

def serialSearh(collection, date):
    result = []
    for i in collection:
        if i["Date"] == date:
            result.append(i)
    return result
        
def ParallelSearching(date, partionedCollection):
    result = []
    workerTask = []
    pool = Pool(processes= 3)
    
    for i in partionedCollection:
        workerTask = pool.apply_async(serialSearh, args=(i,date))
        output = workerTask.get()
        result.append(output)
    
    pool.close()
    return result


In [55]:
# test whether serialSearh is right
serialSearh(partionByTime(MergeCollection)[1], "2017-12-15")

[{'Air Temperature(Celcius)': 18,
  'Date': '2017-12-15',
  'Fire': [{'Confidence': 92,
    'Datetime': '2017-12-15T13:17:17',
    'Latitude': -37.627,
    'Longitude': 149.33,
    'Power': 23.1,
    'SurfaceTemperature': {'Celcius': 42, 'kelvin': 315.9},
    '_FireId': ObjectId('5af3cc31b14a6b797be553cb')},
   {'Confidence': 78,
    'Datetime': '2017-12-15T13:17:17',
    'Latitude': -37.658,
    'Longitude': 149.339,
    'Power': 17.3,
    'SurfaceTemperature': {'Celcius': 36, 'kelvin': 309.8},
    '_FireId': ObjectId('5af3cc31b14a6b797be553cc')},
   {'Confidence': 51,
    'Datetime': '2017-12-15T13:17:17',
    'Latitude': -37.623,
    'Longitude': 149.323,
    'Power': 17.8,
    'SurfaceTemperature': {'Celcius': 38, 'kelvin': 311.9},
    '_FireId': ObjectId('5af3cc31b14a6b797be553cd')},
   {'Confidence': 65,
    'Datetime': '2017-12-15T01:16:23',
    'Latitude': -38.038,
    'Longitude': 142.986,
    'Power': 32.8,
    'SurfaceTemperature': {'Celcius': 40, 'kelvin': 313.4},
    '_Fir

In [56]:
# do paralle searching 
ParallelSearching("2017-12-15", partionByTime(MergeCollection))

[[],
 [{'Air Temperature(Celcius)': 18,
   'Date': '2017-12-15',
   'Fire': [{'Confidence': 92,
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -37.627,
     'Longitude': 149.33,
     'Power': 23.1,
     'SurfaceTemperature': {'Celcius': 42, 'kelvin': 315.9},
     '_FireId': ObjectId('5af3cc31b14a6b797be553cb')},
    {'Confidence': 78,
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -37.658,
     'Longitude': 149.339,
     'Power': 17.3,
     'SurfaceTemperature': {'Celcius': 36, 'kelvin': 309.8},
     '_FireId': ObjectId('5af3cc31b14a6b797be553cc')},
    {'Confidence': 51,
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -37.623,
     'Longitude': 149.323,
     'Power': 17.8,
     'SurfaceTemperature': {'Celcius': 38, 'kelvin': 311.9},
     '_FireId': ObjectId('5af3cc31b14a6b797be553cd')},
    {'Confidence': 65,
     'Datetime': '2017-12-15T01:16:23',
     'Latitude': -38.038,
     'Longitude': 142.986,
     'Power': 32.8,
     'SurfaceTemperature': {'Celcius